In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import * 

from mlxtend.plotting import plot_decision_regions 

pd.options.display.max_columns = 1000

%matplotlib inline

In [2]:
df = pd.read_csv("/data/credit-default.csv")

In [4]:
y = np.where(df.default == 2, 1, 0)
X = pd.get_dummies(df, drop_first=True)
del X["default"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                        test_size = 0.3, random_state =  1)

pipe = pipeline.Pipeline([
    ("scaler", preprocessing.StandardScaler()),
    ("lr", linear_model.LogisticRegression())
])
pipe.fit(X_train, y_train)
print("test accuracy", pipe.score(X_test, y_test))

test accuracy 0.756666666667


In [7]:
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,residence_history,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,4,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,2,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,3,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,4,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,4,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [10]:
df.months_loan_duration.describe()

count    1000.000000
mean       20.903000
std        12.058814
min         4.000000
25%        12.000000
50%        18.000000
75%        24.000000
max        72.000000
Name: months_loan_duration, dtype: float64

In [17]:
pd.cut(df.months_loan_duration, 15)

0        (3.932, 8.533]
1        (44.8, 49.333]
2       (8.533, 13.067]
3        (40.267, 44.8]
4      (22.133, 26.667]
5      (35.733, 40.267]
6      (22.133, 26.667]
7      (35.733, 40.267]
8       (8.533, 13.067]
9        (26.667, 31.2]
10      (8.533, 13.067]
11       (44.8, 49.333]
12      (8.533, 13.067]
13     (22.133, 26.667]
14       (13.067, 17.6]
15     (22.133, 26.667]
16     (22.133, 26.667]
17       (26.667, 31.2]
18     (22.133, 26.667]
19     (22.133, 26.667]
20      (8.533, 13.067]
21       (3.932, 8.533]
22      (8.533, 13.067]
23      (8.533, 13.067]
24      (8.533, 13.067]
25       (3.932, 8.533]
26       (3.932, 8.533]
27      (8.533, 13.067]
28       (3.932, 8.533]
29       (58.4, 62.933]
             ...       
970      (13.067, 17.6]
971    (22.133, 26.667]
972    (22.133, 26.667]
973      (58.4, 62.933]
974      (26.667, 31.2]
975    (22.133, 26.667]
976      (3.932, 8.533]
977      (17.6, 22.133]
978    (22.133, 26.667]
979      (13.067, 17.6]
980      (26.667

In [14]:
df.savings_balance.unique()

array(['unknown', '< 100 DM', '501 - 1000 DM', '> 1000 DM', '101 - 500 DM'], dtype=object)

In [15]:
p1 = 486 / 700
2 * p1 * (1 - p1)

0.4245061224489796

In [47]:
y = np.where(df.default == 2, 1, 0)
X = pd.get_dummies(df, drop_first=True)
del X["default"]

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("fit", tree.DecisionTreeClassifier(random_state=1))
])

param_grid = {
    "fit__max_depth": np.arange(5, 15)
}
gs = model_selection.GridSearchCV(cv=5, estimator=pipe, param_grid=param_grid)
gs.fit(X, y)
print("best params", gs.best_params_, "best score", gs.best_score_)

best params {'fit__max_depth': 7} best score 0.72


In [48]:
model = gs.best_estimator_.steps[2][1]
from sklearn.tree import export_graphviz
export_graphviz(model, out_file = "tree.dot", feature_names = X.columns)

In [49]:
!dot -Tpng tree.dot -o tree.png

In [50]:
!pwd

/Users/abulbasar/workspace/python/machine-learning/Demos1211


In [56]:
pd.DataFrame({"feature": X.columns, "importance": model.feature_importances_}).sort_values("importance", ascending = False)

,feature,importance
9,checking_balance_unknown,0.185833
0,months_loan_duration,0.141514
1,amount,0.130747
4,age,0.061540
15,purpose_car (used),0.039678
24,savings_balance_< 100 DM,0.039221
3,residence_history,0.036585
14,purpose_car (new),0.035785
13,credit_history_repaid,0.033710
12,credit_history_fully repaid this bank,0.027980


In [ ]:
%%time 

y = np.where(df.default == 2, 1, 0)
X = pd.get_dummies(df, drop_first=True)
del X["default"]

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("fit", ensemble.RandomForestClassifier(random_state=1))
])

param_grid = {
    "fit__max_depth": np.arange(5, 15),
    "fit__max_features": np.linspace(0.1, 0.99, 10),
    "fit__n_estimators": np.arange(1, 5) * 30
    
}
gs = model_selection.GridSearchCV(cv=5, estimator=pipe, param_grid=param_grid, verbose=True)
gs.fit(X, y)
print("best params", gs.best_params_, "best score", gs.best_score_)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


In [60]:
model = gs.best_estimator_.steps[2][1]
pd.DataFrame({"feature": X.columns, 
        "importance": model.feature_importances_})\
.sort_values("importance", ascending = False)

,feature,importance
1,amount,0.140351
0,months_loan_duration,0.116257
9,checking_balance_unknown,0.105249
4,age,0.098375
7,checking_balance_< 0 DM,0.045724
2,installment_rate,0.033644
12,credit_history_fully repaid this bank,0.024400
5,existing_credits,0.022132
37,property_real estate,0.020276
3,residence_history,0.019753


In [32]:
y = np.where(df.default == 2, 1, 0)
X = pd.get_dummies(df, drop_first=True)
del X["default"]

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("fit", linear_model.LogisticRegression(random_state=1))
])

param_grid = {
    "fit__C": 10 ** np.linspace(-3, -2, 10)
}
gs = model_selection.GridSearchCV(cv=5, estimator=pipe, param_grid=param_grid)
gs.fit(X, y)
print("best params", gs.best_params_, "best score", gs.best_score_)

best params {'fit__C': 0.0046415888336127772} best score 0.747
